In [1]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [5]:
ticker = yf.Ticker("AAPL")
history = ticker.history(period="max")
latest_date = history.tail(1).index[0]
s_date = input("Enter the period")

In [6]:
BTC_USD = yf.download("BTC-USD", period= s_date)
BTC_USD.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-03-11,39439.968750,40081.679688,38347.433594,38794.972656,38794.972656,26364890465
2022-03-12,38794.464844,39308.597656,38772.535156,38904.011719,38904.011719,14616450657
2022-03-13,38884.726562,39209.351562,37728.144531,37849.664062,37849.664062,17300745310
2022-03-14,37846.316406,39742.500000,37680.734375,39666.753906,39666.753906,24322159070
2022-03-15,39664.250000,39794.628906,38310.210938,39338.785156,39338.785156,23934000868
